In [24]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
from time import time
from get_vector import get_ntt
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("NTT_maxi.bit")
    ipNTT = ol.NTT_0

    

    # Memory allocate
    inBuffer0 = allocate(shape=(1024,), dtype=np.int16)
    
    # ============= Set Kernel Argument ==============
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    #        bit 3  - ap_ready (Read/COR)
    #        bit 7  - auto_restart (Read/Write)
    #        bit 9  - interrupt (Read)
    #        others - reserved
    # 0x04 : Global Interrupt Enable Register
    #        bit 0  - Global Interrupt Enable (Read/Write)
    #        others - reserved
    # 0x08 : IP Interrupt Enable Register (Read/Write)
    #        bit 0 - enable ap_done interrupt (Read/Write)
    #        bit 1 - enable ap_ready interrupt (Read/Write)
    #        others - reserved
    # 0x0c : IP Interrupt Status Register (Read/COR)
    #        bit 0 - ap_done (Read/COR)
    #        bit 1 - ap_ready (Read/COR)
    #        others - reserved
    # 0x10 : Data signal of a
    #        bit 31~0 - a[31:0] (Read/Write)
    # 0x14 : Data signal of a
    #        bit 31~0 - a[63:32] (Read/Write)
    # 0x18 : reserved
    # 0x1c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
    

    
    sum = 0
    for i in range(10):
        NTT_in = open("mq_ntt_in2.txt", "r+")
        NTT_in.seek(0)
        for i in range(1024):
            line = NTT_in.readline()
            inBuffer0[i] = int(line)
        #print(f"input data: {inBuffer0[i]:d}")
        NTT_in.close()
        
        timeKernelStart = time()
    # ================ Data Transfer =================
        logn = 10
        ipNTT.write(0x1C, logn) # Coefficient
        ipNTT.write(0x10, inBuffer0.device_address) # 0x10 bit 31:0
        ipNTT.write(0x14, inBuffer0.device_address) # 0x14 bit 63:32

    # =================== Start ======================
        ipNTT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipNTT.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
        sum += timeKernelEnd - timeKernelStart
        test = get_ntt(inBuffer0)
    
    # ================== Compare =====================
        error_cnt = 0
        NTT_out = open("mq_ntt_out2.txt", "r+")
        NTT_out.seek(0)
        for i in range(1024):
            line = NTT_out.readline()
            golden = int(line)
            if test[i] != golden:
                print(f"Failed, expect value: {golden:d}, got value: {test[i]:d}")
                error_cnt += 1
            #else:
            #    print(f"Calculate value: {test[i]:d}")
        NTT_out.close()
        if error_cnt == 0:
            print("Test Passed!!")
        else:
            print("Test Failed!!")

    print("============================")
    print("Error count : ", error_cnt)
    print("Average time: ", sum/10)
    print("Exit process")

Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
Kernel execution time: 0.0001583099365234375 s
Test Passed!!
Kernel execution time: 0.00012087821960449219 s
Test Passed!!
Kernel execution time: 0.0001163482666015625 s
Test Passed!!
Kernel execution time: 0.00011777877807617188 s
Test Passed!!
Kernel execution time: 0.00011730194091796875 s
Test Passed!!
Kernel execution time: 0.00011587142944335938 s
Test Passed!!
Kernel execution time: 0.00012373924255371094 s
Test Passed!!
Kernel execution time: 0.00011777877807617188 s
Test Passed!!
Kernel execution time: 0.00014495849609375 s
Test Passed!!
Kernel execution time: 0.0001220703125 s
Test Passed!!
Error count :  0
Average time:  0.0001255035400390625
Exit process
